## Etapa 3: Coleta do dataset rotulado

Para conseguirmos treinar um modelo de previsão de sentimentos, precisamos de uma base rotulada, mas conseguir uma base em português, de tweets e dentro do contexto, não é uma tarefa facil! Assim decidimos criar a nossa propria base rotulada, em português e de certa forma contextualizada. Para isso utilizamos os comentarios dos Apps das empresas monitoradas na Google Store, onde tomamos a pressisa de que notas 5 estrelas são comentários positivos e nota 1 estrela comentários negativos. Não utilizamos as notas 2, 3 e 4.

In [1]:
!pip install google-play-scraper -q

You should consider upgrading via the 'c:\users\lbknaak\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


In [33]:
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all
import pandas as pd
import json
import datetime

In [34]:
#Setando a url do app das empresas  

nubank = 'com.nu.production'
banco_inter = 'br.com.intermedium'
c6bank = 'com.c6bank.app'
timeneon = 'br.com.neon'
Bradesco = 'com.bradesco'
santander_br = 'com.santander.app'  
BancodoBrasil = 'br.com.bb.android'   
itau = 'com.itau'
 
NetflixBrasil = 'com.netflix.mediaclient'
PrimeVideoBR = 'com.amazon.avod.thirdpartyclient'
globoplay = 'com.globo.globotv'
DisneyPlusBR = 'com.disney.disneyplus'

submarino = 'com.b2w.submarino'
americanascom = 'com.b2w.americanas'
pontofrio = 'com.sonyericsson.pontofrio'
amazonBR = 'com.amazon.mShop.android.shopping'
CasasBahia = 'com.novapontocom.casasbahia'

CocaCola_Br = ''
PepsiBr = ''
brkindle = ''

Lista_Bancos = [nubank, banco_inter, c6bank, timeneon, Bradesco, santander_br, BancodoBrasil, itau]
Lista_Streamers = [NetflixBrasil, PrimeVideoBR, globoplay, DisneyPlusBR]
Lista_marketplace = [submarino, americanascom, pontofrio, amazonBR, CasasBahia]
#Lista_outros = [CocaCola_Br, PepsiBr, brkindle]

In [35]:
def busca_review_unico(app):
    
    positivos = reviews_all(
    app,
    sleep_milliseconds=0, # defaults to 0
    lang='pt', # defaults to 'en'
    country='br', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    filter_score_with=5 # defaults to None(means all score)
    )
    
    negativos = reviews_all(
    app,
    sleep_milliseconds=0, # defaults to 0
    lang='pt', # defaults to 'en'
    country='br', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    filter_score_with=1 # defaults to None(means all score)
    )
    print('APP : ', app)
    print('Reviews Positivos : ',len(positivos))
    print('Reviews Negativos : ',len(negativos))
    print('-----------------------------------------')
    return positivos, negativos
    

In [36]:
def busca_review_lista(lista_app):
    rev_positivos, rev_negativos = [],[]
    for apps in  lista_app:
        positivos, negativos = busca_review_unico(apps)
        rev_positivos += positivos
        rev_negativos += negativos
    
    print('-----------------------------------------')    
    print('Total Reviews Positivos : ',len(rev_positivos))
    print('Total Reviews Negativos : ',len(rev_negativos))
    print('-----------------------------------------')
    return rev_positivos, rev_negativos

In [37]:
def defaultconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()

In [38]:
def converte_jason_para_dataframe(lista_positiva, lista_negativa):
    rev_positivos_json = json.dumps(lista_positiva, default = defaultconverter)
    rev_negativos_json = json.dumps(lista_negativa, default = defaultconverter)
    
    df_positivos = pd.read_json(rev_positivos_json)
    df_negativos = pd.read_json(rev_negativos_json)
    
    df_positivos=df_positivos[(['content','score'])]
    df_negativos=df_negativos[(['content','score'])]
    
    df_ = pd.concat([df_positivos, df_negativos], ignore_index=True)
    
    return df_

In [39]:
# Capturando os dados da lista de bancos
rev_positivos, rev_negativos = busca_review_lista(Lista_Bancos)

APP :  com.nu.production
Reviews Positivos :  5970
Reviews Negativos :  6965
-----------------------------------------
APP :  br.com.intermedium
Reviews Positivos :  1990
Reviews Negativos :  14726
-----------------------------------------
APP :  com.c6bank.app
Reviews Positivos :  24477
Reviews Negativos :  19303
-----------------------------------------
APP :  br.com.neon
Reviews Positivos :  796
Reviews Negativos :  2786
-----------------------------------------
APP :  com.bradesco
Reviews Positivos :  2786
Reviews Negativos :  11144
-----------------------------------------
APP :  com.santander.app
Reviews Positivos :  597
Reviews Negativos :  12139
-----------------------------------------
APP :  br.com.bb.android
Reviews Positivos :  6766
Reviews Negativos :  5174
-----------------------------------------
APP :  com.itau
Reviews Positivos :  597
Reviews Negativos :  1990
-----------------------------------------
-----------------------------------------
Total Reviews Positivos : 

In [40]:
# Armazenando em um pandas dataset
df_bancos = converte_jason_para_dataframe(rev_positivos, rev_negativos)
df_bancos

,content,score
0,"Gente, mudem pro design anterior. A interface ...",5
1,"Gosto muito desse banco, mas farei uma crítica...",5
2,"Até agora estou gostando, realmente essa nova ...",5
3,Adoro esse APP. Raramente apresenta alguma fal...,5
4,Simplesmente maravilhoso!! Eu nunca vi um banc...,5
...,...,...
118201,O Aplicativo não está enviando SMS para efetua...,1
118202,O APP tá me pedindo uma atualização que não ex...,1
118203,Olá só queria falar que esse aplicativo não es...,1
118204,"O app esta dando erro, travando e nao consigo ...",1


In [41]:
# Exportando pra um CSV
df_bancos.to_csv('reviews_bancos.csv')

In [42]:
# Capturando os dados da lista de Streamers
rev_positivos_stream, rev_negativos_stream = busca_review_lista(Lista_Streamers)

APP :  com.netflix.mediaclient
Reviews Positivos :  8955
Reviews Negativos :  995
-----------------------------------------
APP :  com.amazon.avod.thirdpartyclient
Reviews Positivos :  1791
Reviews Negativos :  1990
-----------------------------------------
APP :  com.globo.globotv
Reviews Positivos :  34427
Reviews Negativos :  8358
-----------------------------------------
APP :  com.disney.disneyplus
Reviews Positivos :  1990
Reviews Negativos :  199
-----------------------------------------
-----------------------------------------
Total Reviews Positivos :  47163
Total Reviews Negativos :  11542
-----------------------------------------


In [43]:
# Armazenando em um pandas dataset
df_streamers = converte_jason_para_dataframe(rev_positivos_stream, rev_negativos_stream)
df_streamers

,content,score
0,"Não tenho do que reclamar sobre o serviço, mas...",5
1,Excelente plataforma...Mas eu opinaria por col...,5
2,Uma observação. Essa última atualização não es...,5
3,Eu acho que até agora é a melhor plataforma de...,5
4,"muito bom, porém as vezes a tela fica atrapalh...",5
...,...,...
58700,Arrumem isso. O app não está carregando. Fica ...,1
58701,Incrível como uma marca tão grande como a Disn...,1
58702,Deixando de assinar o serviço. Além de você pa...,1
58703,"Não gostei, esperava muito mais pelo merchan q...",1


In [44]:
# Exportando pra um CSV
df_streamers.to_csv('reviews_streamers.csv')

In [45]:
# Capturando os dados da lista de marketplace
rev_positivos_marketplace, rev_negativos_marketplace = busca_review_lista(Lista_marketplace)

APP :  com.b2w.submarino
Reviews Positivos :  7363
Reviews Negativos :  5773
-----------------------------------------
APP :  com.b2w.americanas
Reviews Positivos :  398
Reviews Negativos :  17114
-----------------------------------------
APP :  com.sonyericsson.pontofrio
Reviews Positivos :  10230
Reviews Negativos :  2786
-----------------------------------------
APP :  com.amazon.mShop.android.shopping
Reviews Positivos :  1393
Reviews Negativos :  5649
-----------------------------------------
APP :  com.novapontocom.casasbahia
Reviews Positivos :  7363
Reviews Negativos :  2189
-----------------------------------------
-----------------------------------------
Total Reviews Positivos :  26747
Total Reviews Negativos :  33511
-----------------------------------------


In [46]:
# Armazenando em um pandas dataset
df_marketplace = converte_jason_para_dataframe(rev_positivos_marketplace, rev_negativos_marketplace)
df_marketplace

,content,score
0,Superrecomendo! Apesar do aplicativo ainda ser...,5
1,O melhor aplicativo que eu já Abaixei tudo que...,5
2,"Ótimo!! Sem demora para entregar os produtos, ...",5
3,"Perfeito,😍 tive um problema no meu cartão e a ...",5
4,Achei o app maravilhoso!! Ia comprar uma gelad...,5
...,...,...
60253,Não cumpre com o prazo de entrega frustando os...,1
60254,Estou tendo a pior experiência do mundo compre...,1
60255,"Péssimo app e péssima loja, valor sobe a todo ...",1
60256,Experiência péssima. Comprei um guarda roupa v...,1


In [47]:
# Exportando pra um CSV
df_streamers.to_csv('reviews_marketplace.csv')

In [48]:
# Unindo todos os datasets
df_geral = pd.concat([df_bancos, df_streamers, df_marketplace], ignore_index=True)
df_geral

,content,score
0,"Gente, mudem pro design anterior. A interface ...",5
1,"Gosto muito desse banco, mas farei uma crítica...",5
2,"Até agora estou gostando, realmente essa nova ...",5
3,Adoro esse APP. Raramente apresenta alguma fal...,5
4,Simplesmente maravilhoso!! Eu nunca vi um banc...,5
...,...,...
237164,Não cumpre com o prazo de entrega frustando os...,1
237165,Estou tendo a pior experiência do mundo compre...,1
237166,"Péssimo app e péssima loja, valor sobe a todo ...",1
237167,Experiência péssima. Comprei um guarda roupa v...,1


In [49]:
# Verificando se o dataset ficou Balanceado
df_geral.score.value_counts()

1    119280
5    117889
Name: score, dtype: int64

In [30]:
# removendo comentarios para balancear o dataset
df_geral = df_geral[:78000]

In [50]:
# Exportando o dataset Unificado para um CSV
df_geral.to_csv('reviews_geral.csv')